In [48]:
# Reset the schema
import duckdb

# Connect to your DuckDB database (file or in-memory)
conn = duckdb.connect("mtga_local.duckdb")  # or ':memory:'

# Read the SQL file
with open("silver_duckdb_schema.sql", "r") as f:
    sql_script = f.read()

# Execute the entire SQL script
conn.execute(sql_script)

# Optional: check tables
tables = conn.execute("SHOW TABLES").fetchall()
conn.close()
print(tables)


[('decks',), ('dim_cards',), ('matches',), ('players',), ('turn1_hands',)]


In [ ]:
import pandas as pd
import duckdb
import json

# Connect to persistent DuckDB database
conn = duckdb.connect(database='/home/r3gal/develop/mtga_pipeline/cloud/mtga_local.duckdb')

# read each csv 1 game at a time and partition into the correct tables


csv_path = "/home/r3gal/develop/mtga_pipeline/data/parsed_csv/Filtered_Player_20260212_233143_test.csv"


# For each game
# save the deck_list in decks
# save the mulligans
# save the match details in matches
#   card draws -> draw_order

# add a new player to players if no match was found for player_id

# The csv will contain at min 1 game, can contain more

dtype_map = {
    'game_num': 'Int64',
    'timestamp': 'str',
    'event': 'str',
    'payload': 'str'
}
df = pd.read_csv(csv_path, dtype=dtype_map)
df['payload'] = df['payload'].apply(json.loads)

# group by game_num
# for game_num, group_df in df.groupby("game_num"):
#     pass

def insert_player (conn, player_id, display_name, region):
    conn.execute(
        """
        INSERT INTO players (player_id, display_name, region)
        SELECT ?, ?, ?
        WHERE NOT EXISTS (
            SELECT 1 FROM players WHERE player_id = ?
        )
        """,
        (str(player_id), str(display_name), str(region), str(player_id))
    )


# pass

df.head()


,game_num,player_id,timestamp,event,payload
0,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:17 PM,deck_list,"{'id': 'c559b3a9-2778-4b41-9c56-23fceb37520a',..."
1,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:17 PM,GreToClientEvent,"[{'type': 'GREMessageType_DieRollResultsResp',..."
2,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:19 PM,GreToClientEvent,"[{'type': 'GREMessageType_SetSettingsResp', 's..."
3,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:20 PM,GreToClientEvent,"[{'type': 'GREMessageType_GameStateMessage', '..."
4,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:29 PM,GreToClientEvent,"[{'type': 'GREMessageType_GameStateMessage', '..."


In [ ]:

deck_obj = df.iloc[0]['payload'].get('request')
nested = json.loads(deck_obj)
# Gives Deck name
deck_name = nested.get('Summary').get('Name')

# Gets the array of the main deck: cardId and quantity are keys
# [{'cardId': 95192, 'quantity': 9},
#  {'cardId': 93715, 'quantity': 1},
#  {'cardId': 95200, 'quantity': 6}]
deck_list = nested.get('Deck').get('MainDeck')

# sideboard and commander
deck_sideboard = nested.get('Deck').get('Sideboard')
deck_commander = nested.get('Deck').get('CommandZone')

deck_row = pd.DataFrame({
    'deck_id': # auto increment
    'player_id': player_id # pull player_id
    'match_id':  # auto increment
    'deck_list': deck_list
    'deck_sideboard': deck_sideboard
    'deck_commander': deck_commander
})

conn.execute("INSERT INTO players SELECT * FROM df")

In [3]:

con.close()

{'Summary': {'DeckId': 'ded6cdc1-b6f0-4f78-a7dc-228a15d632ea',
  'Mana': '',
  'Name': 'Cat Attack',
  'Attributes': [{'name': 'Version', 'value': '11'},
   {'name': 'TileID', 'value': '93715'},
   {'name': 'LastPlayed', 'value': '"2026-02-12T23:25:44.858186-05:00"'},
   {'name': 'LastUpdated', 'value': '"0001-01-01T00:00:00"'},
   {'name': 'IsFavorite', 'value': 'false'},
   {'name': 'Format', 'value': 'Standard'}],
  'Description': 'Decks/Precon/Precon_EPPFDN_WG_desc',
  'DeckTileId': 93715,
  'DeckArtId': 0,
  'IsCompanionValid': False,
  'FormatLegalities': {},
  'PreferredCosmetics': {'Avatar': '',
   'Sleeve': '',
   'Pet': '',
   'Title': '',
   'Emotes': []},
  'DeckValidationSummaries': [],
  'UnownedCards': {},
  'NetDeckFolderId': None,
  'IsNetDeck': False,
  'checkInbox': False},
 'Deck': {'MainDeck': [{'cardId': 95192, 'quantity': 9},
   {'cardId': 93715, 'quantity': 1},
   {'cardId': 95200, 'quantity': 6},
   {'cardId': 93829, 'quantity': 1},
   {'cardId': 94007, 'quanti

In [42]:
import duckdb
print(duckdb.__version__)


1.4.4
